Script para atualizar a localização dos itens da CCR para as coordenadas informadas no formulário variável

In [7]:
Company.objects.all()

<QuerySet [<Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>, <Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>]>

In [1]:
from tqdm.notebook import tqdm
from django.contrib.gis.geos import LineString, Point
from helpers.histories import bulk_update_with_history

In [2]:
company=Company.objects.get(name='CCR - Rio Sp')
user=User.objects.get(username='rlcs')

In [17]:
reps=Reporting.objects.filter(
        occurrence_type__occurrence_kind='2',
        occurrence_type__name__icontains='Drenagem Profunda Padrão CCR'
).exclude(
    company=company
).only(
    'point',
    'form_data'
)
reps.count(),reps[0].occurrence_type

(3388,
 <OccurrenceType: Drenagem Profunda Padrão CCR - ['CCR - SPVias', 'CCR - Rio Sp', 'CCR - AutoBAn', 'CCR - RodoAnel']>)

In [18]:
lat_keys=['latidudeini','latitudeini','lat','latitude_barreira_nj','latitudedecimal','lat_station','latitude']
lng_keys=['longitudeini','long','longitude_barreira_nj','longitudedecimal','long_station','longitude']
# lat_keys=['latidudeini']
# lng_keys=['longitudeini']

In [19]:
to_fix = []
proper = []

In [20]:
erros=[]
for a in tqdm(reps):
    form_data_lat = None
    form_data_lng = None
    for key in lat_keys:
        if key in a.form_data:
            form_data_lat = a.form_data[key]
            break
    for key in lng_keys:
        if key in a.form_data:
            form_data_lng = a.form_data[key]
            break
    if type(form_data_lng) is str:
        form_data_lng = float(form_data_lng.replace(",", "."))
    if type(form_data_lat) is str:
        form_data_lat = float(form_data_lat.replace(",", "."))
    if type(form_data_lng) is int and '-' in str(form_data_lng) and not '.' in str(form_data_lng):
        form_data_lng = float(str(form_data_lng)[:3]+'.'+str(form_data_lng)[3:])
    if type(form_data_lat) is int and '-' in str(form_data_lat) and not '.' in str(form_data_lat):
        form_data_lat = float(str(form_data_lat)[:3]+'.'+str(form_data_lat)[3:])
    
    if a.point and form_data_lng and form_data_lat:
        if (
        round(a.point[0], 6) != round(form_data_lng, 6) or
        round(a.point[1], 6) != round(form_data_lat, 6)
        ):
            a.point = Point(float(form_data_lng),float(form_data_lat),srid=4326)
            to_fix.append(a)
        else:
            proper.append(a)
    elif not a.point and form_data_lng and form_data_lat:
        a.point = Point(float(form_data_lng),float(form_data_lat),srid=4326)
        to_fix.append(a)
    else:
        erros.append(a)
#     else:
#         erros.append({'number':a.number,"Unidade":a.company.name})

  0%|          | 0/3388 [00:00<?, ?it/s]

In [21]:
len(to_fix),len(proper),len(erros)

(0, 0, 3388)

In [22]:
erros[1].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f88b1929850>,
 'uuid': UUID('29cd6d64-0ae9-475a-b401-760bfa7980e1'),
 'point': <Point object at 0x7f88b1922b50>,
 'form_data': {'zone': '1',
  'format': ['1'],
  'length': 258.0,
  'x_utm_jus': 313308.0,
  'y_utm_jus': 7385716.0,
  'ambient_jus': '1',
  'ambient_mon': '1',
  'diameter_jus': 0.8,
  'cod_lig_monit': '126_OAC2023021k028910',
  'diameter_mont': 0.8,
  'revest_material': ['2'],
  'hole_kind_tubular': ['1'],
  'material_rev_mont': ['2']},
 '_fieldsignals_originals': {(140225472386320,
   140225096618784): {'uuid': UUID('29cd6d64-0ae9-475a-b401-760bfa7980e1'), 'point': <Point object at 0x7f88b1922d90>, 'form_data': {'zone': '1',
    'format': ['1'],
    'length': 258.0,
    'x_utm_jus': 313308.0,
    'y_utm_jus': 7385716.0,
    'ambient_jus': '1',
    'ambient_mon': '1',
    'diameter_jus': 0.8,
    'cod_lig_monit': '126_OAC2023021k028910',
    'diameter_mont': 0.8,
    'revest_material': ['2'],
    'hole_kind_tubular': ['1']

In [19]:
apontamentos = bulk_update_with_history(to_fix,Reporting, use_django_bulk=True,batch_size=30,user=user)

In [36]:
reps.count()

1094

In [21]:
all_rep=to_fix+proper

In [22]:
reps=Reporting.objects.filter(
#         occurrence_type__occurrence_kind='2',
        occurrence_type__name__in=["Terraplenos e Estrutura de Contenção"]
#         ,company = company
).exclude(
    company=company
).exclude(uuid__in=all_rep).values(
    'number','company__name'
)
reps.count()

ValidationError: ["'[CCR - SPVias] CCR-Inv-2024.25938 - 2023-09-30 03:00:00+00:00' não é um UUID válido."]

In [12]:
import pandas as pd
pd.DataFrame(data=reps).to_excel('lat-lng-erros.xlsx')

In [66]:
dado=to_fix[0].form_data
type(dado['lat'])

float

In [33]:
str(dado)[:3]+'.'+str(dado)[3:]

'-23.508046398080900'